In [1]:
import os

In [2]:
%pwd

'd:\\projects\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\projects\\Text-Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigrationManager:
    def __init__ (self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig (
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
        

In [9]:
import os 
import urllib.request as request
import zipfile 
from textSummarizer.logging import logger 
from textSummarizer.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded! with info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [27]:
try: 
    config = ConfigrationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2024-06-03 18:59:05,960: INFO: common: yaml file: config\config.yaml loaded successfully]
[ 2024-06-03 18:59:05,963: INFO: common: yaml file: params.yaml loaded successfully]
[ 2024-06-03 18:59:05,965: INFO: common: created directory at: artifacts]
[ 2024-06-03 18:59:05,967: INFO: common: created directory at: artifacts/data_ingestion]


[ 2024-06-03 18:59:10,208: INFO: 2908722488: artifacts/data_ingestion/samsumdata.zip downloaded! with info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5511:46AE0:1E31AA:28EA35:665DC31F
Accept-Ranges: bytes
Date: Mon, 03 Jun 2024 13:29:07 GMT
Via: 1.1 varnish
X-Served-By: cache-del21745-DEL
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1717421347.402000,VS0,VE305
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 9c1b169c9092fb2214a0e3b407de794308757f37
Expires: Mon, 03 Jun 2024 13:34:07 GMT
Source-Age: 0

]
